In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

matplotlib.use("TkAgg")

remote = mysql.connector.connect(
    host = "msdb.cvyy46quatrs.ap-northeast-2.rds.amazonaws.com",
    port = "3306",
    user = "root",
    password = "Dbsalstjq128!",
    database = "F1_Analysis"
)

cur = remote.cursor()

In [4]:
url = "https://racingnews365.com/height-weight-f1-drivers"
page = urlopen(url)

soup = BeautifulSoup(page, "html.parser")

In [49]:
import re

In [116]:
name = 'Alex Albon'
h = "1.86m/ (6' 1'')"
w = "74kg/163lb"

print((' ').join(re.findall('[A-Z][a-z]+', name)))
print(re.findall('\d.\d+m',h)[0])
print(re.findall('\d+kg',w)[0])

Alex Albon
1.86m
74kg


In [174]:
d = "F1 drivers' height and weight 2025 season"
print(re.findall('\d+',d)[1])

2025


Let's get (driver's name, driver's height, driver's weight) info of each season. (2021~2025)

In [ ]:
content_fields = soup.find_all(class_="content-fields")
for each_content in content_fields:
    panel_list = each_content.find_all(class_="panel")
    for panel in panel_list:
        season = panel.find(class_="panel__header__title content__heading--section")
        if "age" in season.get_text():
            continue
        else:
            this_season = re.findall('\d+', season.get_text())[0] if len(re.findall('\d+', season.get_text())[0]) == 4 else re.findall('\d+', season.get_text())[1]
            driver_list = panel.find_all(class_="panel__content")
            for driver_info in driver_list:
                each_driver = driver_info.find_all("td")                    
                for idx, each in enumerate(each_driver):
                        if idx %3 == 0:
                            each_info = []
                            each_info.append(this_season)
                            name = (' ').join(re.findall('[A-Z][a-z]+', each.get_text().strip())).strip()
                            each_info.append(name)
                                
                        if idx %3 == 1:   
                            height = re.findall('\d.\d+m',each.get_text().strip())[0]
                            height = int(float(height[:-1])*100)
                            each_info.append(height)
                            
                        if idx %3 == 2:  
                            weight = re.findall('\d+kg',each.get_text().strip())[0]    
                            weight = int(weight[:-2])               
                            each_info.append(weight)
                            sql = "insert into driver_info (season, name, height, weight) values (%s, %s, %s, %s)"
                            
                            cur.execute(sql, (each_info[0], each_info[1], each_info[2], each_info[3]))
                            remote.commit()

In [ ]:
remote.close()